# STAC client

## STAC API client
EODAG can perform an item search over a STAC compliant API. Found STAC items are returned as [EOProduct](../../api_reference/eoproduct.rst#eodag.api.product._product.EOProduct) objects with STAC metadata mapped to OGC OpenSearch Extension for Earth Observation.

EODAG comes with already configured providers, but you can also add new ones dynamically.

In [1]:
import os
from eodag import EODataAccessGateway, setup_logging
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
setup_logging(verbose=2)

dag = EODataAccessGateway()

2024-11-05 15:45:36,845 eodag.config                     [INFO    ] Loading user configuration from: /home/sylvain/.config/eodag/eodag.yml
2024-11-05 15:45:36,924 eodag.core                       [INFO    ] Locations configuration loaded from /home/sylvain/.config/eodag/locations.yml


List already configured providers providing a STAC API compliant service:

In [2]:
[p.name for p in dag.providers_config.values() if hasattr(p, "search") and p.search.type == 'StacSearch']

['usgs_satapi_aws',
 'earth_search',
 'earth_search_cog',
 'earth_search_gcs',
 'planetary_computer',
 'hydroweb_next',
 'dedl',
 'geodes']

Then, let's update EODAG's configuration with a new STAC provider (West University of Timisoara STAC catalog) using [add_provider()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.add_provider) method:

In [3]:
dag.add_provider("uvt", "https://stac.sage.uvt.ro/search")

2024-07-12 10:52:57,592 eodag.config                     [INFO    ] uvt: unknown provider found in user conf, trying to use provided configuration


List available product types for this provider (using [product types discovery](https://eodag.readthedocs.io/en/stable/notebooks/api_user_guide/2_providers_products_available.html#Product-types-discovery) in background):

In [4]:
[pt["ID"] for pt in dag.list_product_types("uvt")]

2024-07-12 10:52:57,608 eodag.config                     [INFO    ] Fetching external product types from https://cs-si.github.io/eodag/eodag/resources/ext_product_types.json
2024-07-12 10:53:03,720 eodag.search.qssearch            [INFO    ] Fetching product types: https://stac.sage.uvt.ro/search/../collections


['ancpi-judete-mnt',
 'clc-2018-20',
 'clc-2018-20-vector',
 'copernicus-dem-30',
 'esa-worldcover-2020',
 'esa-worldcover-2021',
 'maja-cams',
 'sentinel-1-grd',
 'sentinel-1-rtc',
 'sentinel-2-l1c',
 'sentinel-2-l1c-ndvi',
 'sentinel-2-l2a-maja']

Search some *sentinel-2-l1c* products over Romania:

In [5]:
prods_S2L1C = dag.search(productType="sentinel-2-l1c", locations=dict(country="ROU"), start="2020-05-01", end="2020-05-15", items_per_page=10, count=True)
prods_S2L1C

2024-07-12 10:53:08,118 eodag.core                       [INFO    ] Searching product type 'sentinel-2-l1c' on provider: uvt
2024-07-12 10:53:08,120 eodag.search.qssearch            [INFO    ] Sending search request: https://stac.sage.uvt.ro/search


SearchResult([EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TET_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TES_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TER_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TDS_20200514T114435, provider=uvt),
              EOProduct(id=S2B_MSIL1C_20200514T094029_N0209_R036_T34TDR_20200514T114435, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UMP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35ULP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TNN_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TMN_20200513T112445, provider=uvt)])

Filter over any item property using crunchers:

In [6]:
from eodag.plugins.crunch.filter_property import FilterProperty

prods_S2L1C_filtered = prods_S2L1C.crunch(FilterProperty({"cloudCover": 10, "operator": "lt"}))
prods_S2L1C_filtered

2024-07-12 10:53:09,723 eodag.crunch.property            [INFO    ] Finished filtering products. 4 resulting products


SearchResult([EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35UMP_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TNN_20200513T112445, provider=uvt),
              EOProduct(id=S2A_MSIL1C_20200513T092031_N0209_R093_T35TMN_20200513T112445, provider=uvt)])

Available assets from the first filtered product

In [7]:
prods_S2L1C_filtered[0].assets

{'B01': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B01.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Band 1 - Coastal aerosol - 60m', 'eo:bands': [{'name': 'B01', 'common_name': 'coastal', 'description': 'Band 1 - Coastal aerosol', 'center_wavelength': 0.443, 'full_width_half_max': 0.027}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [1830, 1830], 'proj:transform': [60.0, 0.0, 499980.0, 0.0, -60.0, 5400000.0, 0.0, 0.0, 1.0]}, 'B02': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B02.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Band 2 - Blue - 10m', 'eo:bands': [{'name': 'B02', 'common_name': 'blue', 'description': 'Band 2 - Blue', 'center_wavelength': 0.49, 'full_width_half_max': 0.098}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [10980, 10980], 'proj:transform': [10.0, 0.0, 499980.0, 0.0, -10.0, 5400000.0, 0.0, 0.0, 1.0]}, 'B03': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B03.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Band 3 - Green - 10m', 'eo:bands': [{'name': 'B03', 'common_name': 'green', 'description': 'Band 3 - Green', 'center_wavelength': 0.56, 'full_width_half_max': 0.045}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [10980, 10980], 'proj:transform': [10.0, 0.0, 499980.0, 0.0, -10.0, 5400000.0, 0.0, 0.0, 1.0]}, 'B04': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B04.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Band 4 - Red - 10m', 'eo:bands': [{'name': 'B04', 'common_name': 'red', 'description': 'Band 4 - Red', 'center_wavelength': 0.665, 'full_width_half_max': 0.038}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [10980, 10980], 'proj:transform': [10.0, 0.0, 499980.0, 0.0, -10.0, 5400000.0, 0.0, 0.0, 1.0]}, 'B05': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B05.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Band 5 - Vegetation red edge 1 - 20m', 'eo:bands': [{'name': 'B05', 'common_name': 'rededge', 'description': 'Band 5 - Vegetation red edge 1', 'center_wavelength': 0.704, 'full_width_half_max': 0.019}], 'proj:bbox': [499980.0, 5290200.0, 609780.0, 5400000.0], 'proj:epsg': 32635, 'proj:shape': [5490, 5490], 'proj:transform': [20.0, 0.0, 499980.0, 0.0, -20.0, 5400000.0, 0.0, 0.0, 1.0]}, 'B06': {'href': 'https://eodata.sage.uvt.ro/Sentinel/Sentinel-2/2020/5/13/T35/S2A_MSIL1C_20200513T092031_N0209_R093_T35UNP_20200513T112445.SAFE/GRANULE/L1C_T35UNP_A025544_20200513T092634/IMG_DATA/T35UNP_20200513T092031_B06.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'Band 6 - Vegetation red edge 2 - 20m', 'eo:bands': [{'name': 'B06', 'common_name': 'rededge', 'description': 'Band 6 - Vegetation red edge 2', 'center_wavelength': 0.74, 'full_width_half_max': 0.018}], 'proj:bbox': [499980

## STAC Static catalog client

EODAG can search for items over a STAC static catalog using [StaticStacSearch](https://eodag.readthedocs.io/en/latest/plugins_reference/generated/eodag.plugins.search.static_stac_search.StaticStacSearch.html) plugin. Path to the catalog (or item) must be set as `search.api_endpoint` and the download plugin can also be set ([HTTPDownload](https://eodag.readthedocs.io/en/latest/plugins_reference/generated/eodag.plugins.download.http.HTTPDownload.html) by default) depending on the provider.

Here is an example with a catalog from https://stacindex.org/catalogs/spot-orthoimages-canada-2005, which will use 
 `HTTPDownload` as download plugin, without credentials as no authentication is needed for download.
 
See [download plugins documentation](../../plugins_reference/download.rst) for other available plugins.

<div class="alert alert-warning">

Warning

Please note that `StaticStacSearch` plugin development is still at an early stage. If search is too slow using this plugin, please use a catalog with less elements.

</div>

In [8]:
# Decrease logging level
setup_logging(verbose=1)

# add the provider
dag.add_provider(
    "stac_http_provider", 
    search={
        "type": "StaticStacSearch", 
        "api_endpoint": "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/catalog.json",
        "timeout": 60,
    },
)

Let's perform search :

In [9]:
from shapely.geometry import Polygon

search_polygon = Polygon([(-70, 45), (-75, 47), (-80, 47), (-80, 44)])
query_args = {
    "provider": "stac_http_provider", 
    "start": "2007-05-01", 
    "end": "2007-05-06" , 
    "geom": search_polygon
}
products = dag.search(**query_args)
products

SearchResult([EOProduct(id=S5_07713_4536_20070505, provider=stac_http_provider),
              EOProduct(id=S5_07702_4605_20070505, provider=stac_http_provider),
              EOProduct(id=S5_07724_4507_20070505, provider=stac_http_provider)])

In [10]:
# plot products and search polygon on map
import ipyleaflet as ipyl

m = ipyl.Map(center=(45, -75), zoom=5)

polygon_layer = ipyl.GeoJSON(data=search_polygon.__geo_interface__, style=dict(color='blue'))
m.add_layer(polygon_layer)

items_layer = ipyl.GeoJSON(data=products.as_geojson_object(), style=dict(color='green'))
m.add_layer(items_layer)
m

Map(center=[45, -75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

Check last product assets

In [13]:
products[-1].assets

{'pan': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07724_4507_20070505/s5_07724_4507_20070505_p10_1_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'pan'}, 'B3': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07724_4507_20070505/s5_07724_4507_20070505_m20_3_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'B3'}, 'B2': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07724_4507_20070505/s5_07724_4507_20070505_m20_2_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'B2'}, 'B4': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07724_4507_20070505/s5_07724_4507_20070505_m20_4_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'B4'}, 'B1': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07724_4507_20070505/s5_07724_4507_20070505_m20_1_lcc00_cog.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'title': 'B1'}, 'thumbnail': {'href': 'https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07724_4507_20070505/s5_07724_4507_20070505_tn.jpg', 'type': 'image/jpeg', 'title': 'thumbnail'}}

Download B1 band:

In [14]:
# create a workspace
workspace = 'eodag_workspace_stac_client'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

path = products[-1].assets["B1"].download(output_dir=workspace)
path

0.00B [00:00, ?B/s]

'/home/sylvain/workspace/eodag/docs/notebooks/tutos/eodag_workspace_stac_client/S5_07724_4507_20070505'

### 